In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import xarray as xr

import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

import seaborn as sns

#### Import compiled_SUPERsnow.nc

In [ ]:
fname = '~/Documents/projects/thesis/results/output/compiled_SUPERsnow.nc'

#ds full
ds = xr.open_dataset(fname,  chunks={'time':1,'x':1000,'y':1000})
ds.close()

#~~~~~~~~~~~~~~ds small
# dsmall = ds.sel(time='2013')
# dsmall.close()

# ds = dsmall
# ds.close()

#~~~~~~~~~~~ds peak
dpeak = ds.isel(time=[0,7,18,30,42,49]) #0,7,22,28,41,49]
dpeak.close()

ds = dpeak
ds.close()
ds

#### Plotting features:

In [ ]:
# legends/labels
d_str = pd.to_datetime(ds['time'].values).strftime("%m-%d-%Y")
print('number of legend labels:', len(d_str))

# fontsize, linewitch, etc.
fs_titles = 24
fs_labels = 24
fs_axes = 20
fs_text = 20
fs_legend = 20

#### FUNCTIONS

In [ ]:
# FUNCTION - standardize lidar by mean
def standard(x):
    a = np.where(x>0) #, x, np.nan)
    return x/np.nanmean(a)

s = xr.apply_ufunc(standard, ds.snow, dask='parallelized', output_dtypes=[np.float32])

In [ ]:
# for i in range(s.time.size):
   
#     plt.imshow(s[i])
#     plt.colorbar()
#     plt.show()

In [ ]:
# FUNCTION - subtracts mean
def minus_mean(x):
    a = np.where(x>0) #, x, np.nan)
    return x - np.nanmean(a)


In [ ]:
# FUNCTION - histogram
def histogram(x):
    '''makes histogram for all values >0 in lidar images'''
    h, bx = np.histogram(x[x !=0], bins=binx)
    return h, bx[:-1]

In [ ]:
# FUNCTION - plot histogram
def step_hist_plt(hist, bin_x, date_iter, axis, alpha=0.4, lw=2, shaded=True):
    
    if shaded is True:
        a.step(bx,h, lw=lw, label = d_str[i])
        a.fill_between(bx,h, step="pre", alpha=alpha)
        
    if shaded is False:
        a.step(bx,h, lw=lw, label = d_str[i])

#### call functions and plot

In [ ]:
# binx = np.arange(0,2500,1) #(start, stop, step by [cm])
binx = np.arange(0,250,0.01)

f=plt.figure(num=0, figsize=(20,10))
a=plt.gca()

with ProgressBar(): #ATTEMPTING TO RESCALE
    for i in range(ds.time.size):
        s = xr.apply_ufunc(standard, ds.snow, dask='parallelized', output_dtypes=[np.float32])
#         mm_s = xr.apply_ufunc(minus_mean, ds.snow, dask='parallelized', output_dtypes=[np.float32])
        h, bx = xr.apply_ufunc(histogram, s[i].values/10, dask='parallelized', output_dtypes=[np.float32])
        step_hist_plt(h, bx, i, a, alpha=0.2, lw=1)

# with ProgressBar(): #SAVED FIGS FROM HERE
#     for i in range(ds.time.size):
#         h, bx = xr.apply_ufunc(histogram, ds.snow[i].values/10, dask='parallelized', output_dtypes=[np.float32])
#         step_hist_plt(h, bx, i, a, alpha=0.2, lw=5, shaded=False)
    
    a.set_title('Snow distribution nearest peak SWE flight date', fontsize=fs_titles)
    a.set_xlim(0,2.5)
    a.set_xlabel('snow depth rescaled by mean',fontsize=fs_axes)
    a.set_ylabel('frequency', fontsize=fs_axes)
    a.legend(prop={'size': fs_legend})
    plt.savefig('/home/meganmason/Documents/projects/thesis/figures/PeakDist_rescaled', dpi=600)
#     plt.show()